In [1]:

from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
import seaborn as sns
import matplotlib.pyplot as plt

# Has multi-dimensional arrays and matrices. Has a large collection of
# mathematical functions to operate on these arrays.
import numpy as np

# Data manipulation and analysis.
import pandas as pd

In [2]:
class BaseAgent(Agent):
    def __init__(self, model, stamina):
        super().__init__(model)
        self.stamina = stamina

    def move_towards(self, target_pos):
        """Move towards the target position (target_pos is a tuple)."""
        x, y = self.pos
        target_x, target_y = target_pos

        # Move horizontally towards the target
        if x < target_x:
            x += 1  # Move right
        elif x > target_x:
            x -= 1  # Move left

        # Move vertically towards the target
        if y < target_y:
            y += 1  # Move down
        elif y > target_y:
            y -= 1  # Move up

        # Update the position on the grid
        self.model.grid.move_agent(self, (x, y))
        print(f"{type(self).__name__} moved to {self.pos}.")

    def move(self):
        """Move randomly to a neighboring cell."""
        # Get all possible neighboring positions
        possible_moves = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
        # Choose a random move from the available positions
        new_pos = self.random.choice(possible_moves)
        # Update the position on the grid
        self.model.grid.move_agent(self, new_pos)
        print(f"{type(self).__name__} moved to {new_pos}.")

    def rest(self):
        """Rest to regain stamina."""
        max_stamina = 100  # Define maximum stamina
        if self.stamina < max_stamina:
            self.stamina = min(self.stamina + 10, max_stamina)
            print(f"{type(self).__name__} resting. Stamina: {self.stamina}/{max_stamina}")

    def step(self):
        """Define agent behavior during each step."""
        if self.model.is_night:
            # Move towards the lifepod if it's night
            lifepod_pos = self.model.lifepod_location
            if self.pos != lifepod_pos:  # If not already at the lifepod
                self.move_towards(lifepod_pos)
            else:
                print(f"{type(self).__name__} resting at lifepod during night at {self.pos}.")
                self.rest()
        else:
            # Define day behavior in derived classes
            print(f"{type(self).__name__} acting during the day.")

In [3]:
class Miner(BaseAgent):
    def __init__(self, model, stamina=69):
        super().__init__(model, stamina)
        self.iron = 0

    def step(self):
        """Perform the miner's actions for each step."""
        if self.model.is_night:  # Check if it's nighttime
            lifepod = self.get_lifepod()
            if lifepod:
                print(f"Miner at {self.pos} moving towards Lifepod at {lifepod.pos} during the night.")
                if self.pos != lifepod.pos:  # If not at the Lifepod, move towards it
                    self.move_towards(lifepod.pos)
                else:
                    print(f"Miner at {self.pos} is staying at the Lifepod during the night.")
        else:  # Daytime actions
            if self.stamina > 0:
                nearest_drill = self.find_nearest_drill()

                if nearest_drill:
                    print(f"Miner at {self.pos} moving towards drill at {nearest_drill.pos}")
                    
                    # Check if miner is within one step of the drill
                    if self.is_near_drill(nearest_drill):
                        print(f"Miner is near the drill at {nearest_drill.pos}. Starting to mine.")
                        self.use_drill(nearest_drill)
                    else:
                        # Move towards the drill
                        self.move_towards(nearest_drill.pos)
                else:
                    print("No drill found.")
                    self.rest()  # If no drill is found, the miner rests
            else:
                self.rest()  # Rest when stamina is depleted

    def is_near_drill(self, drill):
        """Check if the miner is adjacent to the drill (within one step)."""
        return abs(self.pos[0] - drill.pos[0]) <= 1 and abs(self.pos[1] - drill.pos[1]) <= 1

    def find_nearest_drill(self):
        """Find the nearest drill on the grid."""
        drills = [agent for agent in self.model.agents if isinstance(agent, Drill) and not agent.is_broken()]
        print(f"Available drills: {[drill.pos for drill in drills]}")  # Debug: print available drills
        if not drills:
            return None
        # Find the drill with the minimum Manhattan distance
        nearest_drill = min(drills, key=lambda drill: abs(drill.pos[0] - self.pos[0]) + abs(drill.pos[1] - self.pos[1]))
        print(f"Nearest drill found at {nearest_drill.pos}")  # Debug: print nearest drill
        return nearest_drill
        
    def use_drill(self, drill):
        """Use the drill to collect resources."""
        if drill.fuel > 0 and not drill.is_broken():
            print("Drill is functional, starting mining.")
            drill.mine()  # The drill mines and updates its fuel, health, and iron state
            self.iron += 3  # Miner collects iron
            print(f"Mining... Total Iron: {self.iron}")
            
            # After collecting iron, store it in the Lifepod
            lifepod = self.get_lifepod() 
            if lifepod:
                lifepod.store_iron(3)  # Store 3 iron in the Lifepod

            self.stamina -= 1  # Miner consumes stamina each time it mines
        else:
            print("Drill cannot be used due to lack of fuel or health.")
            
    def rest(self):
        """Regain stamina when resting."""
        super().rest()  # Call the base class's rest method
        print(f"Miner at {self.pos} is resting to regain stamina.")  # Print when the miner is resting

    def get_lifepod(self):
        """Retrieve the Lifepod in the model."""
        # Assuming you have only one Lifepod in your model, find it
        for agent in self.model.agents:
            if isinstance(agent, Lifepod):
                return agent
        return None

In [4]:
class Engineer(BaseAgent):
    def __init__(self, model, stamina=69):
        super().__init__(model, stamina)

    def step(self):
        """Define the engineer's actions for each step."""
        if self.model.is_night:  # Follow the BaseAgent's night behavior
            super().step()
            return

        if self.stamina > 0:
            # Check for a nearby broken drill and move towards it if necessary
            nearest_broken_drill = self.find_nearest_broken_drill()
            if nearest_broken_drill:
                print(f"Engineer at {self.pos} moving towards broken drill at {nearest_broken_drill.pos}.")
                if self.is_near_drill(nearest_broken_drill):
                    self.repair(nearest_broken_drill)  # Repair the drill if near
                else:
                    self.move_towards(nearest_broken_drill.pos)  # Move towards the broken drill
            else:
                self.move()  # Move randomly if no broken drill is found
                print(f"Engineer at {self.pos} is moving randomly.")
        else:
            self.rest()  # Rest when stamina is depleted
            print(f"Engineer at {self.pos} is resting to regain stamina.")

    def find_nearest_broken_drill(self):
        """Find the nearest broken drill on the grid."""
        drills = [
            agent for agent in self.model.agents
            if isinstance(agent, Drill) and agent.is_broken()
        ]
        if drills:
            nearest_drill = min(drills, key=lambda drill: abs(self.pos[0] - drill.pos[0]) + abs(self.pos[1] - drill.pos[1]))
            print(f"Nearest broken drill found at {nearest_drill.pos}.")
            return nearest_drill
        print("No broken drills found.")
        return None

    def is_near_drill(self, drill):
        """Check if the engineer is adjacent to the drill (within one step)."""
        return abs(self.pos[0] - drill.pos[0]) <= 1 and abs(self.pos[1] - drill.pos[1]) <= 1

    def repair(self, drill):
        """Repair the drill."""
        if drill.is_broken():
            print(f"Engineer at {self.pos} repairing drill at {drill.pos}.")
            drill.repair()
            self.stamina = max(0, self.stamina - 10)  # Reduce stamina but ensure it doesn't go below 0
        else:
            print(f"Drill at {drill.pos} is not broken.")

    def rest(self):
        """Regain stamina when resting."""
        super().rest()  # Call the base class's rest method
        if self.stamina < 100:
            self.stamina = min(100, self.stamina + 10)  # Increment stamina towards full
            print(f"Engineer at {self.pos} is resting. Current stamina: {self.stamina}.")


In [5]:
class Drill(Agent):
    def __init__(self, model, max_fuel=69, max_health=69):
        super().__init__(model)
        self.fuel = max_fuel
        self.iron = 0
        self.health = max_health

    def is_broken(self):
        """Check if the drill is broken (out of health or fuel)."""
        return self.health <= 0 or self.fuel <= 0  # Drill is considered broken if either fuel or health is zero

    def repair(self):
        """Repair the drill to full health and reset fuel."""
        self.health = 69  # Full health after repair
        self.fuel = 69  # Reset fuel after repair

    def mine(self):
        """Simulate mining using the drill."""
        if not self.is_broken():
            # Perform mining if the drill is functional
            self.fuel -= 1  # Decrease fuel
            self.iron += 3  # Increase iron
            self.health = max(0, self.health - 1)  # Decrease health, ensure it's non-negative
            print(f"Mining... Drill Fuel: {self.fuel}, Drill Health: {self.health}, Iron: {self.iron}")
        else:
            print("Drill is out of fuel or broken!")


In [6]:
class Farmer(BaseAgent):
    def __init__(self, model, stamina=69):
        super().__init__(model, stamina)

    def step(self):
        if self.model.is_night:  # Check if it's nighttime
            lifepod = self.get_lifepod()
            if lifepod:
                print(f"Farmer at {self.pos} moving towards Lifepod at {lifepod.pos} during the night.")
                if self.pos != lifepod.pos:  # If not at the Lifepod, move towards it
                    self.move_towards(lifepod.pos)
                else:
                    print(f"Farmer at {self.pos} is staying at the Lifepod during the night.")
        else:  # Daytime actions
            if self.stamina > 0:
                nearest_greenhouse = self.find_nearest_greenhouse()  # Find the nearest greenhouse
                
                if nearest_greenhouse:
                    # Move towards the greenhouse until the farmer is near it
                    if not self.near_greenhouse(nearest_greenhouse):
                        self.move_towards(nearest_greenhouse.pos)
                        print(f"Farmer at {self.pos} is moving towards greenhouse at {nearest_greenhouse.pos}.")
                    else:
                        self.collect_food(nearest_greenhouse)  # Collect food if near the greenhouse
                else:
                    print("No greenhouse found.")
            else:
                self.rest()  # Rest when stamina is depleted

    def collect_food(self, greenhouse):
        """Collect food from the greenhouse and store in the Lifepod."""
        if greenhouse.food > 0:
            collected_food = 5  # The farmer collects 5 units of food
            greenhouse.food -= collected_food  # Reduce food from the greenhouse
            print(f"Farmer collected {collected_food} food from greenhouse at {self.pos}.")
            
            lifepod = self.get_lifepod()
            if lifepod:
                lifepod.store_food(collected_food)  # Store collected food in the Lifepod

            # Decrease stamina only when collecting food
            self.stamina = max(self.stamina - 5, 0)  # Decrease stamina for collecting food
        else:
            print(f"Greenhouse at {self.pos} has no food left.")

    def find_nearest_greenhouse(self):
        """Find the nearest greenhouse on the grid."""
        greenhouses = [agent for agent in self.model.agents if isinstance(agent, Greenhouse)]
        if greenhouses:
            nearest_greenhouse = min(greenhouses, key=lambda greenhouse: abs(greenhouse.pos[0] - self.pos[0]) + abs(greenhouse.pos[1] - self.pos[1]))
            return nearest_greenhouse
        return None

    def near_greenhouse(self, greenhouse):
        """Check if the farmer is near a specific greenhouse."""
        return self.pos == greenhouse.pos
    
    def get_lifepod(self):
        """Retrieve the Lifepod in the model."""
        for agent in self.model.agents:
            if isinstance(agent, Lifepod):
                return agent
        return None

In [7]:
class Greenhouse(Agent):
    def __init__(self, model, max_food=100, food_add_rate=10):
        super().__init__(model)
        self.food = 0
        self.max_food = max_food
        self.food_add_rate = food_add_rate  # Rate at which food is added to the greenhouse

    def step(self):
        # The greenhouse adds food at a set rate, up to the max capacity
        self.add_food(self.food_add_rate)

    def add_food(self, amount):
        """Add food to the greenhouse, respecting the max_food limit."""
        if self.food + amount <= self.max_food:
            self.food += amount
            print(f"Greenhouse added {amount} food. Total food: {self.food}")
        else:
            self.food = self.max_food
            print(f"Greenhouse food reached max capacity: {self.food}")



In [8]:
class Lifepod(Agent):
    def __init__(self, model):
        super().__init__(model)
        self.food = 20  # Starting food in the Lifepod
        self.iron = 0  # Starting iron in the Lifepod

    def store_iron(self, amount):
        """Store the given amount of iron in the Lifepod."""
        self.iron += amount
        print(f"Lifepod now has {self.iron} iron.")

    def get_iron(self):
        """Get the current amount of iron in the Lifepod."""
        return self.iron

    def store_food(self, amount):
        """Store the given amount of food in the Lifepod."""
        self.food += amount
        print(f"Lifepod now has {self.food} food.")

    def get_food(self):
        """Get the current amount of food in the Lifepod."""
        return self.food

In [9]:
class SurvivalModel(Model):
    """The goddamn model"""
    
    def __init__(self, width, height, seed=69):
        super().__init__(seed=seed)
        self.grid = MultiGrid(width, height, False)
        
        # Set lifepod location at the center of the grid
        self.lifepod_location = (width // 2, height // 2)
        
        # Create and place the Lifepod
        self.lifepod = Lifepod(self)
        self.grid.place_agent(self.lifepod, self.lifepod_location)
        
        # Create and place worker agents at the lifepod location
        self.miner = Miner(self)
        self.engineer = Engineer(self)
        self.farmer = Farmer(self)
        
        for agent in [self.miner, self.engineer, self.farmer]:
            self.grid.place_agent(agent, self.lifepod_location)

        # Set fixed coordinates for the Drill and Greenhouse for simplicity
        self.drill_location = (self.lifepod_location[0] + 4, self.lifepod_location[1] + 4)  # Example: place drill 2 units to the right of the lifepod
        self.greenhouse_location = (self.lifepod_location[0] - 4, self.lifepod_location[1]+4)  # Example: place greenhouse 4 units to the left of the lifepod
        
        # Create and place resource-generating structures at these specific locations
        self.drill = Drill(self)
        self.greenhouse = Greenhouse(self)
        
        # Ensure drill and greenhouse are not placed on the lifepod location
        if self.drill_location != self.lifepod_location:
            self.grid.place_agent(self.drill, self.drill_location)
        
        if self.greenhouse_location != self.lifepod_location:
            self.grid.place_agent(self.greenhouse, self.greenhouse_location)
          
        # Day-night cycle variables
        self.time_step = 0
        self.is_night = False
        
        # Create DataCollector to track model state
        self.datacollector = DataCollector(
            model_reporters={
                "Total Food": lambda m: m.lifepod.food,
                "Total Iron": lambda m: m.lifepod.iron,  # Track the iron count in the Lifepod
                "Drill Health": lambda m: m.drill.health,
                "Drill Fuel": lambda m: m.drill.fuel,
                "Greenhouse Food": lambda m: m.greenhouse.food,
                "Miner Stamina": lambda m: m.miner.stamina,
                "Engineer Stamina": lambda m: m.engineer.stamina,
                "Farmer Stamina": lambda m: m.farmer.stamina,
                "Is Night": lambda m: m.is_night,
                "Cell Contents": self.record_cell_contents
            }
        )

    def record_cell_contents(self):
        """Record the contents of each cell in the grid."""
        cell_data = {}
        for cell_contents, (x, y) in self.grid.coord_iter():
            cell_data[(x, y)] = [type(agent).__name__ for agent in cell_contents]
        return cell_data
    
    def toggle_day_night(self):
        """Toggle the day-night cycle based on the time step."""
        # Day lasts for 30 iterations, night lasts for 15 iterations
        if self.time_step % 45 < 30:
            self.is_night = False
        else:
            self.is_night = True
        print(f"Time Step: {self.time_step}, Night: {self.is_night}")
    
    def step(self):
        """Advance the model by one step."""
        # Update the day-night cycle
        self.toggle_day_night()
        
        # Collect data and advance agents
        self.datacollector.collect(self)
        self.agents.shuffle_do("step")
        
        # Increment the time step
        self.time_step += 1

In [10]:
model = SurvivalModel(10, 10)
for _ in range(100):  
    model.step()

Available drills: [(9, 9)]
Nearest drill found at (9, 9)
Miner at (5, 5) moving towards drill at (9, 9)
Miner moved to (6, 6)
Engineer at (6, 4) is moving randomly.
Farmer moved to (4, 5)
Farmer at (4, 5) is moving towards greenhouse at (1, 5).
Greenhouse added 10 food. Total food: 10
Greenhouse added 10 food. Total food: 20
Available drills: [(9, 9)]
Nearest drill found at (9, 9)
Miner at (6, 6) moving towards drill at (9, 9)
Miner moved to (7, 7)
Farmer moved to (3, 5)
Farmer at (3, 5) is moving towards greenhouse at (1, 5).
Engineer at (7, 5) is moving randomly.
Greenhouse added 10 food. Total food: 30
Available drills: [(9, 9)]
Nearest drill found at (9, 9)
Miner at (7, 7) moving towards drill at (9, 9)
Miner moved to (8, 8)
Farmer moved to (2, 5)
Farmer at (2, 5) is moving towards greenhouse at (1, 5).
Engineer at (8, 5) is moving randomly.
Available drills: [(9, 9)]
Nearest drill found at (9, 9)
Miner at (8, 8) moving towards drill at (9, 9)
Miner is near the drill at (9, 9). Sta

In [11]:
model_data = model.datacollector.get_model_vars_dataframe() 
print(model_data)

    Total Food  Total Iron  Drill Health  Drill Fuel  Greenhouse Food  \
0           20           0            69          69                0   
1           20           0            69          69               10   
2           20           0            69          69               20   
3           20           0            69          69               30   
4           20           3            68          68               40   
..         ...         ...           ...         ...              ...   
95         345         258            52          52              100   
96         350         261            51          51               95   
97         350         264            50          50              100   
98         355         267            49          49               95   
99         360         270            48          48               95   

    Miner Stamina  Engineer Stamina  Farmer Stamina  \
0              69                69              69   
1            